## Signal Processing Fundamentals for Biosignals Demo
### EUNICE Course - 3rd Lecture

This notebook demonstrates the signal processing concepts covered in the lecture
using MNE-Python.


In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.datasets import sample
import os
import pandas as pd
from pathlib import Path

# Setting up plotting parameters
plt.rcParams.update({'font.size': 11})
mne.set_log_level('WARNING')  # Reduce terminal output


### In this demo we will present the basic workflow we follow for the basic analysis of EEG signals

### Load our data and plot the first 10 seconds

In [7]:

# First, we'll load some EEG data and inspect its properties.

# Define data path - replace with your data path or use sample data
# data_path = Path('/path/to/your/data')
# Use MNE sample data for demonstration

data_path = sample.data_path()
sample_data_file = os.path.join(data_path, 'MEG', 'sample', 'sample_audvis_raw.fif')

# Load the raw data
raw = mne.io.read_raw_fif(sample_data_file, preload=True)

# Let's select only EEG channels for simplicity
raw.pick_types(meg=False, eeg=True, eog=True, exclude='bads')

# Display information about the data
print(f"Sampling rate: {raw.info['sfreq']} Hz")
print(f"Number of channels: {len(raw.ch_names)}")
print(f"Duration: {raw.times.max():.2f} seconds")
print(f"Channel names: {raw.ch_names}")

# Plot the first 10 seconds of data
raw.plot(duration=10, n_channels=20, scalings='auto', title='Raw EEG Data')



Sampling rate: 600.614990234375 Hz
Number of channels: 60
Duration: 277.71 seconds
Channel names: ['EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007', 'EEG 008', 'EEG 009', 'EEG 010', 'EEG 011', 'EEG 012', 'EEG 013', 'EEG 014', 'EEG 015', 'EEG 016', 'EEG 017', 'EEG 018', 'EEG 019', 'EEG 020', 'EEG 021', 'EEG 022', 'EEG 023', 'EEG 024', 'EEG 025', 'EEG 026', 'EEG 027', 'EEG 028', 'EEG 029', 'EEG 030', 'EEG 031', 'EEG 032', 'EEG 033', 'EEG 034', 'EEG 035', 'EEG 036', 'EEG 037', 'EEG 038', 'EEG 039', 'EEG 040', 'EEG 041', 'EEG 042', 'EEG 043', 'EEG 044', 'EEG 045', 'EEG 046', 'EEG 047', 'EEG 048', 'EEG 049', 'EEG 050', 'EEG 051', 'EEG 052', 'EEG 054', 'EEG 055', 'EEG 056', 'EEG 057', 'EEG 058', 'EEG 059', 'EEG 060', 'EOG 061']


qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


<mne_qt_browser._pg_figure.MNEQtBrowser(0x57208a980a80) at 0x705ac2600400>

/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


### Handling Bad channels

In [8]:

# Let's mark some channels as bad and then try different rereferencing schemes.

# Mark a few channels as bad for demonstration
raw.info['bads'] = ['EEG 012', 'EEG 015']
print(f"Bad channels: {raw.info['bads']}")

# Plot to see the bad channels
fig = raw.plot(duration=5, n_channels=20, scalings='auto', title='Raw Data with Bad Channels')

# Interpolate bad channels
raw_interp = raw.copy().interpolate_bads()
print("Bad channels interpolated")


Bad channels: ['EEG 012', 'EEG 015']


qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


Bad channels interpolated


/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


### Re-reference the recordings

#### Overview
EEG measures a voltage (difference in electric potential) between each electrode and a reference electrode. This means that whatever signal is present at the reference electrode is effectively subtracted from all the measurement electrodes. Therefore, an ideal reference signal is one that captures none of the brain-specific fluctuations in electric potential, while capturing all of the environmental noise/interference that is being picked up by the measurement electrodes.

In practice, this means that the reference electrode is often placed in a location on the subject’s body and close to their head (so that any environmental interference affects the reference and measurement electrodes similarly) but as far away from the neural sources as possible (so that the reference signal doesn’t pick up brain-based fluctuations). Typical reference locations are the subject’s earlobe, nose, mastoid process, or collarbone. Each of these has advantages and disadvantages regarding how much brain signal it picks up (e.g., the mastoids pick up a fair amount compared to the others), and regarding the environmental noise it picks up (e.g., earlobe electrodes may shift easily, and have signals more similar to electrodes on the same side of the head).

Even in cases where no electrode is specifically designated as the reference, EEG recording hardware will still treat one of the scalp electrodes as the reference, and the recording software may or may not display it to you (it might appear as a completely flat channel, or the software might subtract out the average of all signals before displaying, making it look like there is no reference).

In [9]:
# Compare different referencing schemes

# 1. Original reference
raw_orig = raw_interp.copy()

# 2. Common Average Reference (CAR)
raw_car = raw_interp.copy().set_eeg_reference('average', projection=False)

# 3. Reference to specific electrodes (e.g., mastoids)
# Let's use the channels closest to mastoids for this example
# In real data, you'd use the actual mastoid channels
mastoid_channels = ['EEG 029', 'EEG 030']  # Example channels, adjust based on your montage
raw_mastoids = raw_interp.copy().set_eeg_reference(mastoid_channels, projection=False)

# Plot all three referencing schemes for comparison
channels_to_plot = ['EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005']
start_time = 0
duration = 5

fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
titles = ['Original Reference', 'Common Average Reference', 'Mastoid Reference']
raws = [raw_orig, raw_car, raw_mastoids]

for i, (raw_ref, title) in enumerate(zip(raws, titles)):
    data, times = raw_ref.get_data(picks=channels_to_plot, 
                                   start=int(start_time * raw.info['sfreq']), 
                                   stop=int((start_time + duration) * raw.info['sfreq']), 
                                   return_times=True)
    
    for j, ch_name in enumerate(channels_to_plot):
        axes[i].plot(times, data[j] * 1e6, label=ch_name)
    
    axes[i].set_title(title)
    axes[i].set_ylabel('Amplitude (μV)')
    axes[i].legend(loc='upper right')

axes[2].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()



In [10]:
# Bipolar reference

raw_bip_ref = mne.set_bipolar_reference(raw, anode=["EEG 054"], cathode=["EEG 055"])
raw_bip_ref.plot()

qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


<mne_qt_browser._pg_figure.MNEQtBrowser(0x57208dd33a10) at 0x705ac0e35980>

/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


### Downsampling

EEG and MEG recordings are notable for their high temporal precision, and are often recorded with sampling rates around 1000 Hz or higher. This is good when precise timing of events is important to the experimental design or analysis plan, but also consumes more memory and computational resources when processing the data. In cases where high-frequency components of the signal are not of interest and precise timing is not needed (e.g., computing EOG or ECG projectors on a long recording), downsampling the signal can be a useful time-saver.


In [13]:
# Let's demonstrate downsampling and its effects on the data.

# Original sampling rate
print(f"Original sampling rate: {raw.info['sfreq']} Hz")

# Downsample to 250 Hz
new_freq = 250
raw_downsampled = raw_interp.copy().resample(new_freq)

# Verify new sampling rate
print(f"New sampling rate: {raw_downsampled.info['sfreq']} Hz")

# Compare the signals before and after downsampling
fig, axes = plt.subplots(2, 1, figsize=(15, 8), sharex=True)

# Plot original data
channel = 'EEG 001'  # Choose a channel to plot
data_orig, times_orig = raw_interp.get_data(picks=channel, return_times=True)
axes[0].plot(times_orig, data_orig[0] * 1e6)
axes[0].set_title(f'Original ({raw.info["sfreq"]} Hz)')
axes[0].set_ylabel('Amplitude (μV)')

# Plot downsampled data
data_down, times_down = raw_downsampled.get_data(picks=channel, return_times=True)
axes[1].plot(times_down, data_down[0] * 1e6, 'r')
axes[1].set_title(f'Downsampled ({raw_downsampled.info["sfreq"]} Hz)')
axes[1].set_ylabel('Amplitude (μV)')
axes[1].set_xlabel('Time (s)')

# Show a zoomed section to see the difference in sampling points
zoom_start = 5
zoom_end = 6.2
for ax in axes:
    ax.set_xlim(zoom_start, zoom_end)
    
plt.tight_layout()
plt.show()


Original sampling rate: 600.614990234375 Hz
New sampling rate: 250.0 Hz


By default, MNE-Python resamples using method="fft", which performs FFT-based resampling via scipy.signal.resample(). While efficient and good for most biological signals, it has two main potential drawbacks:

It assumes periodicity of the signal. We try to overcome this with appropriate signal padding, but some signal leakage may still occur.

It treats the entire signal as a single block. This means that in general effects are not guaranteed to be localized in time, though in practice they often are.

Alternatively, resampling can be performed using method="polyphase" instead. In general, using method="polyphase" can also be faster than method="fft" in cases where the desired sampling rate is an integer factor different from the input sampling rate.

In [15]:
# Downsample to 250 Hz
new_freq = raw.info['sfreq']/2.0
raw_downsampled_poly = raw_interp.copy().resample(new_freq, method="polyphase", verbose=True )

# Verify new sampling rate
print(f"New sampling rate: {raw_downsampled_poly.info['sfreq']} Hz")

# Compare the signals before and after downsampling
fig, axes = plt.subplots(2, 1, figsize=(15, 8), sharex=True)

# Plot original data
channel = 'EEG 001'  # Choose a channel to plot
data_orig, times_orig = raw_interp.get_data(picks=channel, return_times=True)
axes[0].plot(times_orig, data_orig[0] * 1e6)
axes[0].set_title(f'Original ({raw.info["sfreq"]} Hz)')
axes[0].set_ylabel('Amplitude (μV)')

# Plot downsampled data
data_down, times_down = raw_downsampled_poly.get_data(picks=channel, return_times=True)
axes[1].plot(times_down, data_down[0] * 1e6, 'r')
axes[1].set_title(f'Downsampled ({raw_downsampled_poly.info["sfreq"]} Hz)')
axes[1].set_ylabel('Amplitude (μV)')
axes[1].set_xlabel('Time (s)')

# Show a zoomed section to see the difference in sampling points
zoom_start = 5
zoom_end = 6.2
for ax in axes:
    ax.set_xlim(zoom_start, zoom_end)
    
plt.tight_layout()
plt.show()

Polyphase resampling neighborhood: ±2 input samples
New sampling rate: 300.3074951171875 Hz


### Data filtering
(more detail in case you are interested can be found on: https://mne.tools/stable/auto_tutorials/preprocessing/25_background_filtering.html)

#### Key Concepts in EEG Filtering

##### Filtering Basics

Filtering is a crucial preprocessing step for EEG data analysis. It alters the spectral content of data by attenuating certain frequencies while preserving others. Common filter types include:

- **High-pass filters**: Remove low frequencies while retaining higher frequencies
- **Low-pass filters**: Remove high frequencies while retaining lower frequencies
- **Band-pass filters**: Preserve frequencies within a specific range
- **Band-stop/notch filters**: Remove frequencies within a specific range (commonly used for power line noise)

##### FIR vs IIR Filters

MNE offers two main filter implementations:

**FIR (Finite Impulse Response) filters:**
- Have a limited-duration impulse response
- Provide better stability and linear phase response
- Recommended for most offline EEG analyses
- Implemented in MNE with the `method='fir'` parameter

**IIR (Infinite Impulse Response) filters:**
- Have an impulse response that can last indefinitely
- More computationally efficient but can have stability issues
- Introduce non-linear phase shifts (phase distortion)
- May be useful for real-time applications
- Implemented in MNE with the `method='iir'` parameter

##### Zero-Phase Filtering

Zero-phase filtering is achieved in MNE by:
- Forward filtering the data
- Reversing the filtered signal
- Filtering again
- Reversing once more

This eliminates phase distortion but doubles the effective filter order and can cause temporal smearing. Use the `phase='zero'` parameter in MNE.

##### Filter Design Considerations

**Filter Order:**
- Higher orders create steeper roll-offs but introduce more temporal smearing
- MNE automatically determines appropriate filter orders by default

**Edge Effects:**
- Filters can create artifacts at signal boundaries
- MNE automatically handles edge artifacts by padding data

**Transition Bandwidth:**
- Specifies the width of the transition between pass and stop bands
- Narrower transitions require higher filter orders

##### Filter Settings Cautions

**High-Pass Filtering:**
- High-pass filters above 0.1 Hz can significantly distort slow ERPs
- Even 0.1 Hz high-pass can alter the shape of late components
- Causal high-pass filters cause signal phase shifts that grow at lower frequencies

**Low-Pass Filtering:**
- Lower low-pass cutoffs can smear sharp response peaks in time
- Designs with narrower transition bands introduce more ringing

##### Practical Recommendations

1. **Filter the continuous data** before epoching to prevent edge artifacts

2. **Apply filters with caution:**
   - High-pass: ≤ 0.1 Hz for ERPs, ~1 Hz for oscillations
   - Low-pass: 40-100 Hz depending on your frequency of interest

3. **Choose appropriate filter parameters:**
   - Use FIR filters with zero-phase for offline analysis
   - Use the 'firwin' design method as a good default option
   - Let MNE automatically determine filter length when possible

4. **Check your data before and after filtering** to ensure the filter is working as expected and not introducing artifacts

Remember that filter settings should always be tailored to your specific research question and reported clearly in your methods.

In [16]:

# Now let's apply different types of filters and see their effects.

# Create copies of the raw data for different filtering operations
raw_filtered = raw_interp.copy()

# 1. High-pass filter at 1 Hz (remove slow drifts)
raw_highpass = raw_filtered.copy().filter(l_freq=1, h_freq=None, 
                                       fir_design='firwin', verbose=False)

# 2. Low-pass filter at 40 Hz (remove high-frequency noise)
raw_lowpass = raw_filtered.copy().filter(l_freq=None, h_freq=40, 
                                      fir_design='firwin', verbose=False)

# 3. Band-pass filter between 1-40 Hz (common EEG preprocessing)
raw_bandpass = raw_filtered.copy().filter(l_freq=1, h_freq=40, 
                                       fir_design='firwin', verbose=False)

# 4. Notch filter at 50 or 60 Hz (power line noise)
raw_notch = raw_filtered.copy().notch_filter(freqs=50, verbose=False)

# Compare all filtering approaches
channel = 'EEG 001'  # Choose a channel to visualize
titles = ['Raw Unfiltered', 'High-pass (>1 Hz)', 'Low-pass (<40 Hz)', 
          'Band-pass (1-40 Hz)', 'Notch (50 Hz)']
raws = [raw_filtered, raw_highpass, raw_lowpass, raw_bandpass, raw_notch]

fig, axes = plt.subplots(len(raws), 1, figsize=(15, 15), sharex=True)

# Plot a 5-second segment
start_time = 0
duration = 5

for i, (raw_plot, title) in enumerate(zip(raws, titles)):
    data, times = raw_plot.get_data(picks=channel, 
                                   start=int(start_time * raw.info['sfreq']), 
                                   stop=int((start_time + duration) * raw.info['sfreq']), 
                                   return_times=True)
    
    axes[i].plot(times, data[0] * 1e6)
    axes[i].set_title(title)
    axes[i].set_ylabel('Amplitude (μV)')

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

#### Let's look at all the recordings (bandpass / highpass)

In [17]:
raw_bandpass.plot()
raw_highpass.plot()

qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass
qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


<mne_qt_browser._pg_figure.MNEQtBrowser(0x572096c5dd90) at 0x705ab6bbb000>

/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


### Channel Locations

In [18]:
raw_filtered.plot_sensors(show_names=True, ch_type="eeg")
plt.show()

### Frequency Domain Analysis

In [19]:
# Let's compute and visualize the power spectral density (PSD) using different methods.

# Compute PSDs using different methods

# 1. Welch's method
fmin, fmax = 0, 30  # Frequency range of interest
raw_psd_welch = raw_bandpass.compute_psd(method='welch', fmin=fmin, fmax=fmax)

# 2. Multitaper method
raw_psd_multitaper = raw_bandpass.compute_psd(method='multitaper', fmin=fmin, fmax=fmax)

# Plot both PSDs for comparison
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot Welch PSD
spectrum = raw_psd_welch.get_data(picks='eeg')
freqs = raw_psd_welch.freqs
for i in range(spectrum.shape[0]):
    axes[0].semilogy(freqs, spectrum[i, :], alpha=0.5, lw=0.5, color='black')
    
mean_spectrum = np.mean(spectrum, axis=0)
axes[0].semilogy(freqs, mean_spectrum, lw=2, color='red')
axes[0].set_title('Power Spectral Density (Welch method)')
axes[0].set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Power (µV²/Hz)')
axes[0].grid(True)

# Color EEG frequency bands
bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma': (30, 45)
}

colors = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#CC79A7']
for (band, freq_range), color in zip(bands.items(), colors):
    # Find closest frequencies in our frequency vector
    idx_min = np.argmin(abs(freqs - freq_range[0]))
    idx_max = np.argmin(abs(freqs - freq_range[1]))
    
    # Mark the frequency band
    axes[0].axvspan(freqs[idx_min], freqs[idx_max], color=color, alpha=0.2, label=band)
    
axes[0].legend(loc='upper right')

# Plot Multitaper PSD
spectrum = raw_psd_multitaper.get_data(picks='eeg')
freqs = raw_psd_multitaper.freqs
for i in range(spectrum.shape[0]):
    axes[1].semilogy(freqs, spectrum[i, :], alpha=0.5, lw=0.5, color='black')
    
mean_spectrum = np.mean(spectrum, axis=0)
axes[1].semilogy(freqs, mean_spectrum, lw=2, color='red')
axes[1].set_title('Power Spectral Density (Multitaper method)')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Power (µV²/Hz)')
axes[1].grid(True)

# Color EEG frequency bands
for (band, freq_range), color in zip(bands.items(), colors):
    # Find closest frequencies in our frequency vector
    idx_min = np.argmin(abs(freqs - freq_range[0]))
    idx_max = np.argmin(abs(freqs - freq_range[1]))
    
    # Mark the frequency band
    axes[1].axvspan(freqs[idx_min], freqs[idx_max], color=color, alpha=0.2, label=band)
    
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.show()


### PSD plots using MNE 

In [22]:
spectrum = raw_highpass.compute_psd(fmin=1, fmax=45)
spectrum.plot(average=True, picks="data", exclude="bads", amplitude=False, )
plt.show()

In [23]:
spectrum.plot_topomap()
plt.show()

In [24]:
spectrum.plot_topo()
plt.show()